In [2]:
import pandas as pd
import shap
import matplotlib.pyplot as plt
import numpy as np

/Users/pascalcsukovits/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('data/finance_stocks/finance_stocks-train.csv', nrows=10)

df.head()

,timestamp,tweet_text,tweet_url,tweet_type,price_of_ticker,change_of_ticker,tickers_mentioned,category
0,2023-11-15T09:36:42.028000+00:00,$GOLD's Massive Range.\n\nIn the past ~year we...,https://twitter.com/user/status/17247221551437...,tweet,['15.71'],['+3.69%'],['$GOLD'],stock_images
1,2023-11-15T06:01:59.788000+00:00,RT @SmartReversals: $SPX - Daily Chart:\n\n✅Ta...,https://twitter.com/user/status/17246687922221...,retweet,['4495.71'],['+1.87%'],['$SPX'],stock_images
2,2023-11-15T06:01:55.590000+00:00,RT @SmartReversals: $NDX - Daily Chart:\n\n✅Ta...,https://twitter.com/user/status/17246687824535...,retweet,['15812.473'],['+2.08%'],['$NDX'],stock_images
3,2023-11-15T06:01:51.329000+00:00,RT @SmartReversals: $IWM - Daily Chart:\n\n✅Ta...,https://twitter.com/user/status/17246687591081...,retweet,['178.46'],['+5.21%'],['$IWM'],stock_images
4,2023-11-15T04:01:49.009000+00:00,RT @coiledspringcap: Everyone has been concern...,https://twitter.com/user/status/17246376898767...,retweet,['4495.71'],['+1.87%'],['$SPX'],stock_images


In [7]:
from transformers import pipeline

In [5]:
pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")


In [8]:
df

,timestamp,tweet_text,tweet_url,tweet_type,price_of_ticker,change_of_ticker,tickers_mentioned,category
0,2023-11-15T09:36:42.028000+00:00,$GOLD's Massive Range.\n\nIn the past ~year we...,https://twitter.com/user/status/17247221551437...,tweet,['15.71'],['+3.69%'],['$GOLD'],stock_images
1,2023-11-15T06:01:59.788000+00:00,RT @SmartReversals: $SPX - Daily Chart:\n\n✅Ta...,https://twitter.com/user/status/17246687922221...,retweet,['4495.71'],['+1.87%'],['$SPX'],stock_images
2,2023-11-15T06:01:55.590000+00:00,RT @SmartReversals: $NDX - Daily Chart:\n\n✅Ta...,https://twitter.com/user/status/17246687824535...,retweet,['15812.473'],['+2.08%'],['$NDX'],stock_images
3,2023-11-15T06:01:51.329000+00:00,RT @SmartReversals: $IWM - Daily Chart:\n\n✅Ta...,https://twitter.com/user/status/17246687591081...,retweet,['178.46'],['+5.21%'],['$IWM'],stock_images
4,2023-11-15T04:01:49.009000+00:00,RT @coiledspringcap: Everyone has been concern...,https://twitter.com/user/status/17246376898767...,retweet,['4495.71'],['+1.87%'],['$SPX'],stock_images
5,2023-11-15T04:01:44.817000+00:00,RT @coiledspringcap: #SPX equal weight breakin...,https://twitter.com/user/status/17246376783927...,retweet,['4495.71'],['+1.87%'],['$SPX'],stock_images
6,2023-11-15T04:01:40.263000+00:00,RT @TriggerTrades: $SPX remained above 4375 th...,https://twitter.com/user/status/17246376379723...,retweet,['4495.71'],['+1.87%'],['$SPX'],stock_images
7,2023-11-15T02:11:47.180000+00:00,"RT @TrendSpider: $SPY, $QQQ and $AAPL all prin...",https://twitter.com/user/status/17246108137746...,retweet,"['385.33', '187.44', '448.73']","['+2.11%', '+1.41%', '+1.9%']","['$QQQ', '$AAPL', '$SPY']",stock_images
8,2023-11-15T02:11:41.182000+00:00,$NET \n\nWas waiting on Cloudflare and it erup...,https://twitter.com/user/status/17246106454321...,tweet,['70.61'],['+9.52%'],['$NET'],stock_images
9,2023-11-15T01:06:39.739000+00:00,Currently at $2860--Documenting everything in ...,https://twitter.com/user/status/17245942794969...,quote tweet,['29.63'],['+6.85%'],['$AI'],stock_images


In [9]:
predictions = [pipe(text)[0]['label'] for text in df["tweet_text"]]
df['Sentiment'] = predictions

In [13]:
shap_df = df[['tweet_text', 'Sentiment']]
shap_df

,tweet_text,Sentiment
0,$GOLD's Massive Range.\n\nIn the past ~year we...,positive
1,RT @SmartReversals: $SPX - Daily Chart:\n\n✅Ta...,neutral
2,RT @SmartReversals: $NDX - Daily Chart:\n\n✅Ta...,neutral
3,RT @SmartReversals: $IWM - Daily Chart:\n\n✅Ta...,neutral
4,RT @coiledspringcap: Everyone has been concern...,negative
5,RT @coiledspringcap: #SPX equal weight breakin...,positive
6,RT @TriggerTrades: $SPX remained above 4375 th...,positive
7,"RT @TrendSpider: $SPY, $QQQ and $AAPL all prin...",neutral
8,$NET \n\nWas waiting on Cloudflare and it erup...,neutral
9,Currently at $2860--Documenting everything in ...,neutral


In [14]:
explainer = shap.Explainer(pipe)
shap_values = explainer(shap_df["tweet_text"])

PartitionExplainer explainer: 11it [02:16, 13.69s/it]                        


In [15]:
shap_values

.values =
array([array([[ 0.00000000e+00, -2.18786299e-05,  1.61486239e-03],
              [ 0.00000000e+00, -2.18786299e-05,  1.61486239e-03],
              [ 0.00000000e+00, -2.18786299e-05,  1.61486239e-03],
              [ 0.00000000e+00, -2.18786299e-05,  1.61486239e-03],
              [ 0.00000000e+00, -2.18786299e-05,  2.82883919e-03],
              [ 0.00000000e+00, -2.18786299e-05,  2.82883919e-03],
              [ 0.00000000e+00, -2.18786299e-05,  2.82883919e-03],
              [ 0.00000000e+00, -2.18786299e-05,  2.31153172e-03],
              [ 0.00000000e+00, -2.18786299e-05,  5.93582968e-04],
              [ 0.00000000e+00, -2.18786299e-05,  5.93582968e-04],
              [ 0.00000000e+00, -2.18786299e-05,  5.93582968e-04],
              [ 0.00000000e+00, -2.18786299e-05,  5.93582968e-04],
              [ 0.00000000e+00, -2.18786299e-05,  5.93582968e-04],
              [ 0.00000000e+00, -2.18786299e-05,  5.93582968e-04],
              [ 0.00000000e+00, -2.18786299e-05,  5.

In [18]:
shap.plots.text(shap_values[:, :, "positive"])